In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import psycopg2

from datetime import date


In [ ]:
# Function to Get the current
# working directory
def current_path():
    print("Current working directory before")
    print(os.getcwd())
    print()
 
 
# Driver's code
# Printing CWD before
current_path()
 
# Changing the CWD
os.chdir('../../../Work Order - Data/WO_Folders/WO_2022-2023')
 
# Printing CWD after
current_path()

In [ ]:
cwd = os.getcwd()  # Get the current working directory (cwd)
file = os.listdir(cwd)[0]  # Get all the files in that directory
print("Files in %r: %s" % (cwd, file))

In [ ]:
os.path.exists(file)

In [ ]:
df = pd.read_csv(file)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df['Date Open'] = pd.to_datetime(df['Created On']).dt.date
df['Date Closed'] = pd.to_datetime(df['Completed']).dt.date #Creates new column and sets data type from DateTime -> Date

In [ ]:
df['Date Difference'] = date.today() - df['Date Open'] #Takes the difference between today's date from Date Open
df['Date Difference'] = df['Date Difference'].dt.days #Converts DateTime Type to Integer

In [ ]:
df['Duration'] = df['Date Closed'] - df['Date Open'] #Takes the diffrence between Date Closed from Date Open
df['Duration'] = df['Duration'].dt.days #Converts DateTime Type to Integer
df['Duration'] = df['Duration'].abs() #Gets the Absolute Value of Column Values

In [ ]:
df['Site_ID'] = df['Source Site'].str[-3:] #Takes the last 3 characters froms string in this case, we get extract the Site Code

In [ ]:
df['School Year'] = '2022 - 2023'

In [ ]:
df['Duration'] = df['Duration'].fillna(df['Date Difference']) #Fill null values from Duration column with values from Date Difference

In [ ]:
df['Row Count'] = 1

In [ ]:
df = df.drop(columns=['Area/Room #', 'Originator', 'Originator First Name', 'Work requested', 'Cause', 'Action Taken', 'Comments' ,'Longitude', 'Latitude', 'PM Title', 'Completed','Created On', 'Originated']) #Drops/Removes the columns from table

In [ ]:
df = df[df['Site_ID'].str.contains("ZZ-|032|941") == False]

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df = df.drop_duplicates(subset='Work Order #', keep='first')
df.info()

In [ ]:
clean_tbl_name = file.lower().replace(" ","").replace("?", "")\
                    .replace("-", "_").replace(r"/","_").replace("\\", "_")\
                    .replace("%", "").replace(")", "").replace(r"(","").replace("$", "")\
                    .replace("pm", "").replace("1", "").replace("2", "").replace("3", "")\
                    .replace("4", "").replace("5", "").replace("6", "").replace("7", "")\
                    .replace("8", "").replace("9", "").replace("0", "").replace("_", "")\
                    .replace("workorder", "updated_work_orders")
clean_tbl_name

In [ ]:
df.columns = [x.lower().replace(" ","_").replace("?", "").replace("#", "id")\
                    .replace("-", "_").replace(r"/","_").replace("\\", "_")\
                    .replace("%", "").replace(")", "").replace(r"(","").replace("$", "") for x in df.columns]

df.columns

In [ ]:
df.dtypes

In [ ]:
replacements = {
    'object': 'varchar',
    'float64': 'float',
    'int64': 'int',
    'datetime64': 'timestamp',
    'timedelta64[ns]': 'varchar'
}

replacements

In [ ]:
col_str = ", ".join("{} {}".format(n, d) for (n,d) in zip(df.columns, df.dtypes.replace(replacements)))
col_str

In [ ]:
conn_string = "host=####\
               dbname='#####'\
               user=##### password='####'"

conn = psycopg2.connect(conn_string)
cursor = conn.cursor()
print('opened database successfully!')

In [ ]:
#drop table
cursor.execute("drop table if exists work_orders;")

In [ ]:
#create table
cursor.execute("CREATE TABLE work_orders \
(work_order_id INT NOT NULL PRIMARY KEY, wo_status VARCHAR(50), priority VARCHAR(50), \
work_category VARCHAR(50) REFERENCES work_category(work_category), problem VARCHAR(50), \
source_site VARCHAR(50), source_location VARCHAR(50), source_user VARCHAR(50), work_type VARCHAR(50),\
labor_hrs float, part_cost float, labor_cost float, total_hrs float, grand_total float, \
date_open DATE, date_closed DATE, \
date_difference float, duration float, site_id INT REFERENCES schools(site_id), school_year VARCHAR(25), row_count INT NOT NULL)")

In [ ]:
df.to_csv('updated_work_orders.csv', header=df.columns, index=False, encoding='utf=8')

my_file = open('updated_work_orders.csv')
print('File opened in memory and export successful!')

#upload to db

SQL_STATEMENT = """
COPY work_orders FROM STDIN WITH 
    CSV
    HEADER
    DELIMITER AS ','
"""

cursor.copy_expert(sql = SQL_STATEMENT, file=my_file)
print('file copied to db!')

In [ ]:
cursor.execute("grant select on table work_orders to public")
conn.commit()

cursor.close()
print('CSV imported to DB successfully!')